<a href="https://colab.research.google.com/github/TGaston/dashboutou/blob/master/Stockscreener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun  8 21:16:34 2020

@author: 15145
"""

# -*- coding: utf-8 -*-
"""
Created on Sat May  9 15:02:41 2020

@author: 15145
"""
import requests
from bs4 import BeautifulSoup 
import csv
import pandas as pd
import numpy as np
import pickle
#mostActiveStocksUrl = 'https://www.nasdaq.com/markets/most-active.aspx&quot';
 
mostActiveStocksUrl = 'https://ca.finance.yahoo.com/most-active';
r= requests.get(mostActiveStocksUrl)
data=r.text
soup=BeautifulSoup(data)
 
 
t=[]
for a in soup.find_all('a', href=True):
    t.append(a['href'])

 
df=pd.DataFrame({'brute':t})

df["Indexes"]= pd.Series(t).str.find('?').to_frame()

df= df[df['Indexes']>-1]
df= df[df['Indexes']<20]
df=df.reset_index()
stocks=[]
for i in range(0,len(df)):
    a= 3+df["Indexes"].loc[[i]]
    a=a.item()
    b=df["brute"].loc[[i]].str.len().item()
    c=df["brute"].loc[[i]].str.slice(a,b)
    c=c.item()
    stocks.append(c)
 
temp = []
 
for item in stocks:
    if item.find(".TO") != -1 and len(item)<8:
        temp.append(item)







mostActiveStocksUrl = 'https://finance.yahoo.com/most-active';
r= requests.get(mostActiveStocksUrl)
data=r.text
soup=BeautifulSoup(data)
 
 
t=[]
for a in soup.find_all('a', href=True):
    t.append(a['href'])

 
df=pd.DataFrame({'brute':t})

df["Indexes"]= pd.Series(t).str.find('?').to_frame()

df= df[df['Indexes']>-1]
df= df[df['Indexes']<20]
df=df.reset_index()
stocks=[]
for i in range(0,len(df)):
    a= 3+df["Indexes"].loc[[i]]
    a=a.item()
    b=df["brute"].loc[[i]].str.len().item()
    c=df["brute"].loc[[i]].str.slice(a,b)
    c=c.item()
    stocks.append(c)
 
temp2 = []
 
for item in stocks:
    if item.find("=") == -1 and item.find("^") == -1 and len(item)<5:
        temp2.append(item)
 
with open('stocks', 'wb') as f:
    pickle.dump(temp, f)
    
with open('stocksUS', 'wb') as f:
    pickle.dump(temp2, f)

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun May 17 18:40:17 2020

@author: 15145
"""

import datetime 
startdate=datetime.datetime(2020, 9, 2)
enddate=datetime.datetime(2020, 12, 28)

import pandas as pd
import pandas_datareader as pdr

import pickle
#aapl = pdr.get_data_yahoo('AAPL', start=datetime.datetime(2006, 10, 1), end=datetime.datetime(2012, 1, 1))
with open('stocks', 'rb') as f:
    tickers = pickle.load(f)
with open('stocksUS', 'rb') as f:
    tickersUS = pickle.load(f)

old_tickers=['FTS.TO', 'TRP.TO','COTY','APHA.TO','OGI']

tickers=list(dict.fromkeys([*tickers,*tickersUS,*old_tickers]))
 
def get(tickers, startdate, enddate):
  def data(ticker):
    return (pdr.get_data_yahoo(ticker, start=startdate, end=enddate))
  datas = map (data, tickers)
  return(pd.concat(datas, keys=tickers, names=['Ticker', 'Date']))

#tickers = ['AAPL', 'MSFT', 'IBM', 'GOOG']
all_data = get(tickers,startdate , enddate)

Tickers=[]
Date=[]
for i in range(0,len(all_data)):
    t=all_data.index[i][0]
    Tickers.append(t)
    d=all_data.index[i][1]
    Date.append(d)
all_data['Tickers']=Tickers
#all_data['Date']=Date


#all_data=all_data.drop(['Date'], axis=1)

df = all_data.pivot_table(index=['Date'], columns='Tickers', values='Close', aggfunc=sum)
df.to_csv('C:/Users/15145/Desktop/algoT/most_active_stock7.csv')
 

In [ ]:
import pandas as pd
import numpy as np
import talib
df = pd.read_csv("C:/Users/15145/Desktop/algoT/most_active_stock7.csv") 
from zipline.api import order_target, record, symbol

def bolinger_band_ind(stock,df):
    stockd = np.array(df[stock])
    stock_bband = talib.BBANDS(stockd, timeperiod=40, nbdevup=2, nbdevdn=2, matype=0)
#IF STOCK PRICE IS EQUAL OR LOWER THAN LOWERBLIMIT WE GO LONG
    order=a = [0 for x in range(len(stockd))]
    for i in range(0,len(stockd)):
        if stockd[i] <= stock_bband[2][i]:
            order[i]="BUY"
#EXIT IF STOCK PRICE IS EQUAL OR GREATER THAN MIDDLE LINE
        if stockd[i] >= stock_bband[1][i]:
            order[i]="BUY"
    df[stock+'_bol_order']=order
    return df
tickers=list(df.columns.values)
tickers.pop(0) 
for item in tickers:
    bolinger_band_ind(item,df)
df=df.loc[:, (df != 0).any(axis=0)]
df.to_csv('ma_stock_wsignals7.csv')

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon May 18 14:25:26 2020

@author: 15145
"""


import pandas as pd
 
df = pd.read_csv("C:/Users/15145/Desktop/algoT/ma_stock_wsignals4.csv") 

#import subprocess
# Do something with analytics
# ...
# Now call docxfactory script
#subprocess.call(['python2', 'docxcript.py'])


#df_agg = df.pivot_table(index=['Date'], columns='Tickers', values='Close', aggfunc=sum)


#df=df.loc[:, (df != 0).any(axis=0)]


dfe=df.tail(40)
dfe1=dfe.loc[:,pd.DataFrame(dfe.apply(pd.Series.value_counts)).T['BUY']>0]
dfe2=dfe1.loc[:, (dfe1.tail(1) != 'SELL').any(axis=0)]



list_sig=dfe2.columns.values.tolist()


l=[]
for item in list_sig:
    p_index = item.find('_')
    s = item[:p_index]
    l.append(s)




t=[]
for item in old_tickers:
    z='^'+item
    ll=dfe.filter(regex=z,axis=1).columns.values.tolist()
    t+=ll

list_fin=['Date']+t+list_sig+l
 
  
df_e_f=df[list_fin].tail(40).reset_index()

df_e_f.to_csv('C:/Users/15145/Desktop/algoT/liste_stock4.csv',index=False)


In [ ]:
 
import matplotlib.pyplot as plt
import pandas as pd
#import pandas.io.data as web
import numpy as np
import scipy.optimize as sco
import pandas_datareader as pdr


availStocks = ['IAG.TO', 'APHA.TO', 'AC.TO', 'BNS.TO', 'EFN.TO']
start = '2020-02-01'
end = '2020-05-20'


def getStockQuotes(symbols, startDate, endDate):
    quotes = pd.DataFrame()
    
    for symbol in symbols:
        quotes[symbol] = pdr.get_data_yahoo(symbol, start=startDate, end=endDate)['Adj Close']
        
    return quotes
    
def calcPortfolioPerf(weights, meanReturns, covMatrix):
    '''
    Calculates the expected mean of returns and volatility for a portolio of 
    assets, each carrying the weight specified by weights
    
    INPUT
    weights: array specifying the weight of each asset in the portfolio
    meanReturns: mean values of each asset's returns
    covMatrix: covariance of each asset in the portfolio
    
    OUTPUT
    tuple containing the portfolio return and volatility
    '''    
    #Calculate return and variance
    portReturn = np.sum( meanReturns*weights )
    portStdDev = np.sqrt(np.dot(weights.T, np.dot(covMatrix, weights)))
    
    return portReturn, portStdDev
    
def negSharpeRatio(weights, meanReturns, covMatrix, riskFreeRate):
    '''
    Returns the negated Sharpe Ratio for the speicified portfolio of assets
    
    INPUT
    weights: array specifying the weight of each asset in the portfolio
    meanReturns: mean values of each asset's returns
    covMatrix: covariance of each asset in the portfolio
    riskFreeRate: time value of money
    '''
    p_ret, p_var = calcPortfolioPerf(weights, meanReturns, covMatrix)
    
    return -(p_ret - riskFreeRate) / p_var
    
def getPortfolioVol(weights, meanReturns, covMatrix):
    '''
    Returns the volatility of the specified portfolio of assets
    
    INPUT
    weights: array specifying the weight of each asset in the portfolio
    meanReturns: mean values of each asset's returns
    covMatrix: covariance of each asset in the portfolio
    
    OUTPUT
    The portfolio's volatility
    '''
    return calcPortfolioPerf(weights, meanReturns, covMatrix)[1]
    
def findMaxSharpeRatioPortfolio(meanReturns, covMatrix, riskFreeRate):
    '''
    Finds the portfolio of assets providing the maximum Sharpe Ratio
    
    INPUT
    meanReturns: mean values of each asset's returns
    covMatrix: covariance of each asset in the portfolio
    riskFreeRate: time value of money
    '''
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple( (0,1) for asset in range(numAssets))
    
    opts = sco.minimize(negSharpeRatio, numAssets*[1./numAssets,], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    
    return opts
    
def findMinVariancePortfolio(meanReturns, covMatrix):
    '''
    Finds the portfolio of assets providing the lowest volatility
    
    INPUT
    meanReturns: mean values of each asset's returns
    covMatrix: covariance of each asset in the portfolio
    '''
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple( (0,1) for asset in range(numAssets))
    
    opts = sco.minimize(getPortfolioVol, numAssets*[1./numAssets,], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    
    return opts
    
def findEfficientReturn(meanReturns, covMatrix, targetReturn):
    '''
    Finds the portfolio of assets providing the target return with lowest
    volatility
    
    INPUT
    meanReturns: mean values of each asset's returns
    covMatrix: covariance of each asset in the portfolio
    targetReturn: APR of target expected return
    
    OUTPUT
    Dictionary of results from optimization
    '''
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    
    def getPortfolioReturn(weights):
        return calcPortfolioPerf(weights, meanReturns, covMatrix)[0]
    
    constraints = ({'type': 'eq', 'fun': lambda x: getPortfolioReturn(x) - targetReturn},
                   {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0,1) for asset in range(numAssets))
    
    return sco.minimize(getPortfolioVol, numAssets*[1./numAssets,], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    
def findEfficientFrontier(meanReturns, covMatrix, rangeOfReturns):
    '''
    Finds the set of portfolios comprising the efficient frontier
    
    INPUT
    meanReturns: mean values of each asset's returns
    covMatrix: covariance of each asset in the portfolio
    targetReturn: APR of target expected return
    
    OUTPUT
    Dictionary of results from optimization
    '''
    efficientPortfolios = []
    for ret in rangeOfReturns:
        efficientPortfolios.append(findEfficientReturn(meanReturns, covMatrix, ret))
        
    return efficientPortfolios
    



stocks = availStocks
#stocks = ['GOOGL', 'TM', 'KO', 'PEP']
#stocks = [availStocks[i] for i in np.random.random_integers(0,len(availStocks)-1, 3)]
numAssets = len(stocks)



#Retrieve data from online
data = getStockQuotes(stocks, start, end)

#Load data from disk
#data = pd.read_hdf('./stockData.h5', 'quotes').select(lambda x: x in stocks, 1)

#Calculate returns
riskFreeRate = 0.0021
dur = 20
numPeriodsAnnually = 252.0/dur
windowedData = data[::dur]
rets = np.log(windowedData/windowedData.shift(1))

#Calculate stock mean-variance
meanDailyReturn = rets.mean()
covariance = rets.cov()

#Create a new figure
plt.figure(figsize=(8,6))

#Run MC simulation of numPortfolios portfolios
numPortfolios = 100000
results = np.zeros((3,numPortfolios))

#Calculate portfolios
for i in range(numPortfolios):
    #Draw numAssets random numbers and normalize them to be the portfolio weights
    weights = np.random.random(numAssets)
    weights /= np.sum(weights)
    
    #Calculate expected return and volatility of portfolio
    pret, pvar = calcPortfolioPerf(weights, meanDailyReturn, covariance)
    
    #Convert results to annual basis, calculate Sharpe Ratio, and store them
    results[0,i] = pret*numPeriodsAnnually
    results[1,i] = pvar*np.sqrt(numPeriodsAnnually)
    results[2,i] = (results[0,i] - riskFreeRate)/results[1,i]
    
#Plot results of MC on portfolio weights
plt.scatter(results[1,:], results[0,:], c=results[2,:], marker='o')

#Find efficient frontier, annual target returns of 9% and 16% are converted to
#match period of mean returns calculated previously
targetReturns = np.linspace(0.09, 0.26, 50)/(252./dur)
efficientPortfolios = findEfficientFrontier(meanDailyReturn, covariance, targetReturns)
plt.plot([p['fun']*np.sqrt(numPeriodsAnnually) for p in efficientPortfolios], targetReturns*numPeriodsAnnually, marker='x')

#Find portfolio with maximum Sharpe ratio
maxSharpe = findMaxSharpeRatioPortfolio(meanDailyReturn, covariance, riskFreeRate)
rp, sdp = calcPortfolioPerf(maxSharpe['x'], meanDailyReturn, covariance)
plt.plot(sdp*np.sqrt(numPeriodsAnnually), rp*numPeriodsAnnually, 'r*', markersize=15.0)

#Find portfolio with minimum variance
minVar = findMinVariancePortfolio(meanDailyReturn, covariance)
rp, sdp = calcPortfolioPerf(minVar['x'], meanDailyReturn, covariance)
plt.plot(sdp*np.sqrt(numPeriodsAnnually), rp*numPeriodsAnnually, 'y*', markersize=15.0)
    
plt.grid(True)
plt.xlabel('Expected Volatility')
plt.ylabel('Expected Return')
plt.colorbar(label='Sharpe Ratio')
plt.title('Portfolios of Many Assets')
plt.tight_layout()
plt.savefig('Monte Carlo Simulation for Portfolio', dpi=100)

#plt.figure(figsize=(8,6))
ind = np.arange(numAssets)
width = 0.35
fig, ax = plt.subplots(figsize=(8,6))
rects1 = ax.bar(ind, maxSharpe['x'], width, color='r', alpha=0.75)
rects2 = ax.bar(ind + width, minVar['x'], width, color='b', alpha=0.75)
ax.set_ylabel('Weight of Asset in Portfolio')
ax.set_ylim(0,0.6)
ax.set_title('Comparison of Portfolio Compositions')
ax.set_xticks(ind + width)
ax.set_xticklabels(stocks)
plt.tight_layout()
ax.legend((rects1[0], rects2[0]), ('Max Sharpe Ratio', 'Minimum Volatility'))
plt.savefig('Portfolio Compositions', dpi=100)
plt.show()
